## Intel image classification using Xception model

In [1]:
import tensorflow as tf
from keras import layers
from keras.datasets import mnist
from keras.layers import Dropout
from tensorflow import keras

import matplotlib.pyplot as plt

2022-04-09 19:53:25.980229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-09 19:53:25.980259: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import os, shutil, pathlib

original_dir = pathlib.Path("../dataset/intel_images")
new_base_dir = pathlib.Path("../dataset/intel_images_small")

In [3]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

2022-04-09 19:53:39.471821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-09 19:53:39.471870: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-09 19:53:39.471903: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linux): /proc/driver/nvidia/version does not exist
2022-04-09 19:53:39.472283: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory=new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    directory=new_base_dir / "valid",
    image_size=(180, 180),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    directory=new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 6000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [5]:
for data_batch, labels_batch in train_dataset:
    print("Data batch shape: ", data_batch.shape)

    print("Labels batch shape: ", labels_batch.shape)
    break

Data batch shape:  (32, 180, 180, 3)
Labels batch shape:  (32,)


In [6]:
conv_base = tf.keras.applications.Xception(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)

83697664/83683744 [==============================] - 34s 0us/step


In [7]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 89, 89, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 89, 89, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                           

In [9]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []

    for images, labels in dataset:
        preprocessed_images = keras.applications.xception.preprocess_input(images)
        features = conv_base(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

In [10]:
train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

2022-04-09 20:18:38.258188: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32444416 exceeds 10% of free system memory.
2022-04-09 20:18:38.821099: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32444416 exceeds 10% of free system memory.
2022-04-09 20:18:38.876980: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32444416 exceeds 10% of free system memory.
2022-04-09 20:18:38.888109: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62005248 exceeds 10% of free system memory.
2022-04-09 20:18:38.967185: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 62005248 exceeds 10% of free system memory.


In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(6, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
    )


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
    filepath="../models/checkpoints/intel__image_feature_extraction.keras",
    save_best_only=True,
    monitor="val_loss")
]